In [ ]:
import requests
api_key = 'la clef api'
adresse = "URL du Gophish"
serveur = "URL du serveur SMTP"
admin = "m.hayem@rubisenergie.com" # Adresse mail de l'admin. S'il y en a plusieurs, les mettre tous dans une liste

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText



In [ ]:
# Fonction qui va nous permettre de récupérer les résultats d'une campagne donnée
def resultat(id_camp):
    # La fonction marche en mettant l'id de la campagne que l'on veut tester, par exemple: resultat(10)
    
    # Les informations importantes comme la clef API, l'adresse du Gophish, l'adresse du serveur mail et l'email de l'admin sont stockées autre part.
    api_camp = adresse+'/api/campaigns/'+str(id_camp)

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.get(api_camp, headers=headers, verify=False)
        response.raise_for_status()  # Pour lever une exception si erreur HTTP
        data = response.json()

        # Si la campagne est terminée, exécuter le code. Sinon, ne rien faire.
        completed = data.get("completed_date", "")        
        if "0001-01-01" in completed or completed == "":
            print(f"La campagne '{data.get('name')}' est toujours en cours.")
        else:
            print(f"La campagne '{data.get('name')}' s'est terminée le {completed}.")
            
            # Initialisation des variables vides qui vont nous servir plus tard
            compteur_user, compteur_click, compteur_submit = 0,0,0
            clique_liste, submit_liste = [], []
            
            # Début de la boucle qui va parcourir tous les utilisateurs ayant participé à la campagne
            for result in data.get('results', []):
                compteur_user += 1 # On compte combien de personnes ont participé au total
                
                if result['status'] == "Clicked Link": # On regarde d'abord les utilisateurs ayant cliqué sur le lien du faux mail de phishing
                    compteur_click += 1 # On compte combien de personnes ont cliqué sur le lien
                    clique_liste.append(f"{result['first_name']} {result['last_name']}: {result['email']}") # Liste contenant tous les utilisateurs ayant cliqué sur le lien
                    
                    # Envoi du mail à toutes les personnes ayant cliqué sur le lien
                    msg = MIMEMultipart()
                    msg['From'] = 'rubis@rubisenergie.com' # Adresse bateau, on peut mettre ce qu'on veut mais il faut que ça soit un @rubisenergie.com
                    msg['To'] = result['email']
                    msg['Subject'] = 'Campagne anti-phishing' 
                    
                    # Corps du mail en html, pour l'instant c'est moche mais on verra plus tard ce qu'on doit mettre exactement
                    message = f"""
                    <html>
                    <body style="font-family: Arial, sans-serif; color: #333;">
                        <img src="https://www.rubisenergie.com/uploads/2021/07/logo-rubis.png" alt="Logo" width="150"><br><br>
                        <h2 style="color: #0055A4;">Bonjour {result['first_name']} {result['last_name']},</h2><br><br>
                        <p>Au cours d'une campagne anti-phishing, nous (le service informatique) avions envoyé de faux emails à l'ensemble de l'entreprise pour voir comment vous réagiriez à cette menace. <br>
                        Malheureusement, vous n'avez pas réussi le test car vous avez cliqué sur le lien de cet email pourtant douteux et devez donc vous former grâce à un module.
                        Vous pouvez trouver ce module <a href="https://module_phishing.zw">ici</a>.</p> <br><br>
                        
                        N'hésitez pas à nous contacter si vous avez le moindre doute sur un email. <br><br>
                        
                        Bonne journée,<br>
                        Le service informatique de RubisEnergie
                    </body>
                    </html>
                    """
                
                    msg.attach(MIMEText(message, 'html'))
                    mailserver = smtplib.SMTP(serveur, 25)
                    mailserver.sendmail(msg['From'], msg['To'], msg.as_string())
                    mailserver.quit()
                    
                elif result['status'] == "Submitted Data": # On regarde ensuite les utilisateurs qui ont renseigné des données sur le faux mail de phishing
                    compteur_submit += 1 # On compte combien de personnes ont renseigné des données
                    submit_liste.append(f"{result['first_name']} {result['last_name']}: {result['email']}") # Liste contenant tous les utilisateurs ayant renseigné des données
                    
                    # Envoi du mail à toutes les personnes ayant renseigné des données
                    msg = MIMEMultipart()
                    msg['From'] = 'rubis@rubisnergie.com' # Adresse bateau, on peut mettre ce qu'on veut mais il faut que ça soit un @rubisenergie.com
                    msg['To'] = result['email']
                    msg['Subject'] = 'Campagne anti-phishing' 
                    
                    # Corps du mail en html, pour l'instant c'est moche mais on verra plus tard ce qu'on doit mettre exactement
                    message = """
                        <html>
                        <body style="font-family: Arial, sans-serif; color: #333;">
                            Bonjour, <br><br>
 
                            Vous avez récemment soumis vos identifiants lors d’une campagne de phishing organisée par le service informatique, <br><br>
                            
                            Votre présence est donc obligatoire à la session de sensibilisation qui se tiendra « … », 6 place des degrés, Tour Landscape, salle XX au -1.<br><br>
                            
                            Nous vous remercions. 
                        </body>
                        </html>
                        """
                
                    msg.attach(MIMEText(message, 'html'))
                    mailserver = smtplib.SMTP(serveur, 25)
                    mailserver.sendmail(msg['From'], msg['To'], msg.as_string())
                    mailserver.quit()

            # Pour que ça formate bien les listes contenant les coupables dans le message envoyé à l'admin
            clique_liste = "<br>".join(clique_liste) 
            submit_liste = "<br>".join(submit_liste)
                    
            # Conclusions envoyées à l'admin par mail
            msg = MIMEMultipart()
            msg['From'] = 'rubis@rubisenergie.com' # Adresse bateau, on peut mettre ce qu'on veut mais il faut que ça soit un @rubisenergie.com
            msg['To'] = admin # Adresse mail de l'admin
            msg['Subject'] = 'Campagne anti-phishing: Résultats' 
            
            # Corps du mail en html
            message = f"""
                <html>
                <body style="font-family: Arial, sans-serif; color: #333;">
                    <img src="https://www.rubisenergie.com/uploads/2021/07/logo-rubis.png" alt="Logo" width="150"><br><br>
                    {compteur_click} utilisateurs sur {compteur_user} ont cliqué sur le lien durant la campagne '{data.get('name')}'. <br>
                    Voici la liste des personnes ayant cliqué sur un lien:<br><br>
                    
                    {clique_liste}<br><br>
                    
                    
                    {compteur_submit} utilisateurs sur {compteur_user} ont renseigné des données durant la campagne {data.get('name')}.
                    Voici la liste des personnes ayant soumis des données:<br><br>
                    
                    {submit_liste}
                </body>
                </html>
                """
            
            msg.attach(MIMEText(message, 'html'))
            mailserver = smtplib.SMTP(serveur, 25)
            mailserver.sendmail(msg['From'], msg['To'], msg.as_string())
            mailserver.quit()
            
            print(f"Résultats envoyés pour la campagne '{data.get('name')}' !")
            
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
    except ValueError:
        print("Erreur de parsing JSON")